# A Hybrid WiFi/Bluetooth RSS Dataset with LS-based localization algorithm

In [164]:
# Imports
import pandas as pd
import numpy as np
import pprint
import math

In [165]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    scenario_dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes


: 

In [166]:
def calculate_smallest_dataset(dataframes):
    """
    Calculates the minimum over all dataframes' length.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    minimum_length (int): an integer which value represents the minimum over all dataframes' length.
    """

    sizes = []

    for anchor in dataframes:
        df = dataframes[anchor]
        sizes.append(len(df))

    return min(sizes)
        

: 

In [167]:
def calculate_target_anchors_distance(dataframes):
    """
    Calculates the distance between the target and the anchors using the Euclidean norm.
    This is the implementation of (1) on the reference paper.

    Parameters:
    dataframes (dict): a dict which keys are the anchor names and values are the respective dataframes.

    Returns:
    target_anchors_distance (list): Returns a list of distances between the target and the anchor.
    """

    target_anchors_distance = []

    for anchor in dataframes:
        df = dataframes[anchor]
        anchor_coords = [eval(i) for i in df["Relative Coordinates [m]"][0].split(", ")]
        target_coords = [eval(i) for i in df["Target Coordinates [m]"][0].split(", ")]
        target_anchors_distance.append(math.dist(anchor_coords, target_coords))

    # print(target_anchors_distance)
    return target_anchors_distance

# LINKS
# https://www.w3schools.com/python/ref_math_dist.asp
# https://www.w3schools.com/python/ref_string_split.asp
# https://www.geeksforgeeks.org/python-converting-all-strings-in-list-to-integers/


In [168]:
def calculate_mean_values(scenario_dataframe, length): 
    """
    Calculates the mean value over a given amount of bursts.
    This is the implementation of (3) on the reference paper.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    length (int): an integer which value represents the number of data to read in the dataset.

    Returns:
    mean_values (list): Returns a list of mean values over the given amount of bursts.
    """
    
    mean_values = []

    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    bursts = length // 10
    burst_size = length // bursts

    # print("length: ", length, ", bursts: ", bursts, ", burst_size: ", burst_size)

    start = 0
    for i in range(bursts):
        # end = start + burst_size if i < bursts - 1 else length - 1
        end = start + burst_size
        # print("start: ", start, ", end: ", end)
        # print("power_values: ", power_values[start:end])
        mean_values.append(round(np.mean(power_values[start:end]), 4))
        start = end

    if (start < length):
        end = length
        # print("start: ", start, ", end: ", end)
        mean_values.append(round(np.mean(power_values[start:end]), 4))
        start = end
    
    return mean_values

: 

In [169]:
def calculate_target_anchors_estimation(p_initial, n_value, mean_values):
    """
    Calculates the distance estimates between each anchor and the target.
    This is the implementation of (4) on the reference paper.

    Parameteres:
    p_initial (int): The initial RSS value for the distance estimation equation.
    n_value (int): The n_value for the distance estimation formula.
    mean_values (list): A list which values are mean values over the given amount of bursts.

    Returns:
    estimated_distances (list): Returns a list which values are the distance estimations.
    """
    
    estimated_distances = []
    index = 0
    
    for mean in mean_values:
        num = mean - p_initial
        den = 10 * n_value
        estimated_distances.append(10**(num/den))
        index = index + 1

    return estimated_distances


: 

In [170]:
def calculate_A_matrix():
    pass

: 

In [171]:
def calculate_b_vector():
    pass

In [172]:
def calculate_W_matrix():
    pass

In [173]:
def calculate_WLS_output():
    pass

In [174]:
def calculate_target_position_estimation():
    pass

In [191]:
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"
anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]

mean_values = {}
estimated_distances = {}
configurations = {}

p_initial = -40
n_value = 4

dataframes = read_data(anchors, directory_path)
length = calculate_smallest_dataset(dataframes)

for anchor in dataframes:
    
    mean_values[anchor] = calculate_mean_values(dataframes[anchor], length)
    estimated_distances[anchor] = calculate_target_anchors_estimation(p_initial, n_value, mean_values[anchor])
    
    # print("--- ", anchor, " ---")
    # print("")
    # print("Mean values: ", mean_values[anchor])
    # print("")
    # print("Estimated distances:", estimated_distances[anchor])
    # print("")
    # print("-----")
    # print("")

for index in range(len(mean_values["Anchor 1"])):
    i = index + 1
    configurations[i] = {}
    configurations[i]["Mean Power"] = []
    configurations[i]["Estimated Distance"] = []
    configurations[i]["Actual Distance"] = []
    for anchor in dataframes:
        configurations[i]["Mean Power"].append(mean_values[anchor][index])
        configurations[i]["Estimated Distance"].append(round(estimated_distances[anchor][index], 3))
        configurations[i]["Actual Distance"].append(dataframes[anchor]["Distance Target - Anchor [m]"][0])

pprint.pprint(configurations)

{1: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Estimated Distance': [0.16, 0.118, 0.202, 0.132, 0.146, 0.325],
     'Mean Power': [-71.8, -77.1, -67.8, -75.2, -73.4, -59.5]},
 2: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Estimated Distance': [0.149, 0.134, 0.209, 0.127, 0.147, 0.318],
     'Mean Power': [-73.1, -74.9, -67.2, -75.9, -73.3, -59.9]},
 3: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Estimated Distance': [0.154, 0.138, 0.194, 0.184, 0.152, 0.318],
     'Mean Power': [-72.5, -74.4, -68.5, -69.4, -72.7, -59.9]},
 4: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Estimated Distance': [0.166, 0.127, 0.193, 0.14, 0.157, 0.29],
     'Mean Power': [-71.2, -75.8, -68.6, -74.1, -72.2, -61.5]},
 5: {'Actual Distance': [15.994, 15.994, 10.55, 15, 12.944, 12.944],
     'Estimated Distance': [0.167, 0.131, 0.204, 0.154, 0.154, 0.309],
     'Mean Power': [-71.1, -75.3, -67.6, -72.5,

: 